In [34]:
using LinearAlgebra


# helper functions to use in iteration matrix
jacobi_M(A) = inv(diagm(diag(A)))
gauss_M(A) = inv(tril(A))


# Perform Gauss-Seidel or Jacobi iteration on matrix A
function lin_iter(; mthd, A, b, stop, xi=false, tol=1.0e-7)
    if !xi
        xi = [0. for i in 1:size(A,1)]
    end

    M = mthd(A)
    T = I - M*A
    c = M*b

    n = 0
    nb = norm(b,1)

    # iteration equation and remainder equation
    f(x) = T*x + c
    r(x) = b - A*x

    # stopping criteria boolean check helper function
    diff(xi) = norm(f(xi)-xi, 1) > tol
    ratio(xi) = norm(r(xi), 1)/nb > tol

    # which stopping criteria to use
    if stop == "r"
        bool = ratio
    else
        bool = diff
    end

    while bool(xi)
        xi = f(xi)
        n += 1
    end

    return round.(xi,digits=8), n
end


# compare methods
function lin_compare(A, b)
    @time soln_rj, nrj = lin_iter(mthd=jacobi_M, A=A, b=b, stop="r")
    @time soln_dj, ndj = lin_iter(mthd=jacobi_M, A=A, b=b, stop="d")

    @time soln_rg, nrg = lin_iter(mthd=gauss_M, A=A, b=b, stop="r")
    @time soln_dg, ndg = lin_iter(mthd=gauss_M, A=A, b=b, stop="d")

    @time soln = A\b

    jerr = norm(soln-soln_rj, 1)
    gerr = norm(soln-soln_rg, 1)
    
    println("\nstopping criteria: ratio")
    println("jacobi # iter: $(nrj)")
    println("gauss # iter: $(nrg)\n")

    println("stopping criteria: diff")
    println("jacobi # iter: $(ndj)")
    println("gauss # iter: $(ndg)\n")

    println("jacobi error: $(jerr)")
    println("gauss error: $(gerr)\n")
    
    println("jacobi solution: $(soln_rj)")
    println("gauss solution: $(soln_rg)")
    println("actual solution: $(round.(soln, digits=8))")
end

lin_compare (generic function with 1 method)

In [57]:
b = [1., 0., 4.]
A = [3. -1. 1.; 3. 6. 2.; 3. 3. 7.]

lin_compare(A, b)

  0.000049 seconds (101 allocations: 10.328 KiB)
  0.000028 seconds (122 allocations: 12.625 KiB)
  0.000013 seconds (67 allocations: 6.688 KiB)
  0.000012 seconds (80 allocations: 8.109 KiB)
  0.000009 seconds (4 allocations: 416 bytes)

stopping criteria: ratio
jacobi # iter: 20
gauss # iter: 12

stopping criteria: diff
jacobi # iter: 20
gauss # iter: 12

jacobi error: 3.2280701604825435e-8
gauss error: 9.280701747382825e-8

jacobi solution: [0.03508772, -0.23684212, 0.65789472]
gauss solution: [0.03508777, -0.23684209, 0.65789471]
actual solution: [0.03508772, -0.23684211, 0.65789474]


In [62]:
b = [2., -1., 4.]
A = [1. -1. 0.; 2. 2. 3.; -1. 3. 2.]

lin_compare(A, b)


# norm.(eigvals(I - jacobi_M(A)*A))
# norm.(eigvals(I - gauss_M(A)*A))

  0.000519 seconds (9.62 k allocations: 1.027 MiB)
  0.000642 seconds (12.02 k allocations: 1.283 MiB)
  0.000059 seconds (499 allocations: 53.938 KiB)
  0.000058 seconds (675 allocations: 73.188 KiB)
  0.000005 seconds (4 allocations: 416 bytes)

stopping criteria: ratio
jacobi # iter: 2399
gauss # iter: 120

stopping criteria: diff
jacobi # iter: 2399
gauss # iter: 131

jacobi error: Inf
gauss error: 1.3599999988400668e-6

jacobi solution: [-9.344299036280862e307, Inf, 1.7491334310289707e308]
gauss solution: [-12.00000049, -14.00000045, 17.00000042]
actual solution: [-12.0, -14.0, 17.0]


In [65]:
b = [4., -4., 0.]
A = [-2. 1. 0.5; 1. -2. -0.5; 0. 1. 2.]

lin_compare(A, b)

  0.000037 seconds (193 allocations: 20.391 KiB)
  0.000023 seconds (257 allocations: 27.391 KiB)
  0.000011 seconds (75 allocations: 7.563 KiB)
  0.000011 seconds (95 allocations: 9.750 KiB)
  0.000006 seconds (4 allocations: 416 bytes)

stopping criteria: ratio
jacobi # iter: 43
gauss # iter: 14

stopping criteria: diff
jacobi # iter: 47
gauss # iter: 15

jacobi error: 1.981818185115003e-7
gauss error: 3.272727272518594e-7

jacobi solution: [-1.45454553, 1.45454553, -0.72727268]
gauss solution: [-1.4545456, 1.45454533, -0.72727267]
actual solution: [-1.45454545, 1.45454545, -0.72727273]


In [67]:
b = [0., 5., -2., 6.]
A = [2. 1. 0. 0.; -1. 3. 3. 0.; 2. -2. 1. 4.; -2. 2. 2. 5.]

lin_compare(A, b)

# norm.(eigvals(I - jacobi_M(A)*A))
# norm.(eigvals(I - gauss_M(A)*A))

  0.000684 seconds (7.38 k allocations: 806.359 KiB)
  0.000902 seconds (9.22 k allocations: 1008.156 KiB)
  0.000663 seconds (6.99 k allocations: 764.219 KiB)
  0.000783 seconds (8.74 k allocations: 954.969 KiB)
  0.000005 seconds (4 allocations: 464 bytes)

stopping criteria: ratio
jacobi # iter: 1839
gauss # iter: 1743

stopping criteria: diff
jacobi # iter: 1840
gauss # iter: 1743

jacobi error: Inf
gauss error: Inf

jacobi solution: [-3.7803715216214356e306, 1.1078020098449819e308, -1.3527267642236994e308, 4.74861716488516e307]
gauss solution: [-1.8972322531518542e307, -6.94289450128895e307, Inf, -1.282064337846569e307]
actual solution: [-0.61702128, 1.23404255, 0.22695035, 0.36879433]


In [68]:
b = [0., 5., 0., 6., -2., 6.]
A = [4. -1. 0. -1. 0. 0.;
    -1. 4. -1. 0. -1. 0.;
    0. -1. 4. 0. 0. -1.;
    -1. 0. 0. 4. -1. 0.;
    0. -1. 0. -1. 4. -1;
    0. 0. -1. 0. -1. 4.]

lin_compare(A, b)

  0.000034 seconds (149 allocations: 18.484 KiB)
  0.000063 seconds (197 allocations: 24.484 KiB)
  0.000021 seconds (87 allocations: 10.828 KiB)
  0.000031 seconds (115 allocations: 14.328 KiB)
  0.000005 seconds (4 allocations: 656 bytes)

stopping criteria: ratio
jacobi # iter: 32
gauss # iter: 17

stopping criteria: diff
jacobi # iter: 35
gauss # iter: 19

jacobi error: 8.299999992855689e-7
gauss error: 4.999999995147775e-7

jacobi solution: [0.99999992, 1.99999977, 0.99999992, 1.99999984, 0.99999988, 1.99999984]
gauss solution: [0.99999986, 1.99999988, 0.99999995, 1.99999991, 0.99999993, 1.99999997]
actual solution: [1.0, 2.0, 1.0, 2.0, 1.0, 2.0]
